In [35]:
import tensorflow as tf
import tensorflow.keras as tfk
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
from tensorflow.keras.datasets import mnist

In [36]:
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))
trainY = tf.keras.utils.to_categorical(trainY)
testY = tf.keras.utils.to_categorical(testY)

In [54]:
def prepare_data(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(X, Y, n_folds):
    scores = []
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    for train_x, test_x in kfold.split(X):
        model = create_model()
        trainX, trainY, testX, testY = X[train_x], Y[train_x], X[test_x], Y[test_x]
        fit = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        t, acc = model.evaluate(testX, testY, verbose=1)
        scores.append(acc)
    return scores

In [55]:
trainX1, testX1 = prepare_data(trainX, testX)
scores = evaluate_model(trainX1, trainY, 5)

375/375 [==============================] - 1s 3ms/step - loss: 0.0810 - accuracy: 0.9784


In [56]:
np.mean(scores)

Accuracy: mean=97.880 std=0.154, n=5
